House price prediction with R

In [ ]:
#Load required library
require(data.table)
require(stringr)
require(lubridate)
require(zoo)
library(neuralnet)
library(nnet)
#for parallel processing
#install.packages("doMC")
library(doMC)
registerDoMC(3) 
getDoParWorkers() 
getwd()
setwd("/Users/mohameddhameemm/Downloads/ForRTraining")

#library(readxl)
#train_xl <- read_excel("train_sg_ori.xlsx")
#colnames(train_xl)
#load the data
trainDF <- read.csv("train_sg_ori.csv", header = TRUE)
test <- read.csv("test_sg_ori.csv", header = TRUE)
#colnames(train)
#str(train)
#summary(train)
#train$SUnitName <- as.numeric(as.character(train$SUnitName))
#sapply(train,mode)
#summary(train$SUnitName)
#train_foratted <- transform(train, char = as.numeric(train$SUnitName))

trainDF <- trainDF[,c("IRemainingLease","IPplStayinginFlatTypeinthisTown","IPopulationDensityinthisTown",
                      "IAgeofFlat","IBedRooms","IFloorAreaSqm", "DResalePrice")]

test <- test[,c("IRemainingLease","IPplStayinginFlatTypeinthisTown","IPopulationDensityinthisTown",
                "IAgeofFlat","IBedRooms","IFloorAreaSqm")]

#Storing for Scaling back the predictions..
train_o <- trainDF

## check the train data
#summary(train$SUnitName) # CLEAN
summary(trainDF$IAgeofFlat) # CLEAN
summary(trainDF$IBedRooms)# CLEAN
summary(trainDF$IFloorAreaSqm)
summary(trainDF$IRemainingLease)
summary(trainDF$IPplStayinginFlatTypeinthisTown)
summary(trainDF$IPopulationDensityinthisTown)

## check teh test data
#summary(test$SUnitName) # CLEAN
summary(test$IAgeofFlat) # CLEAN
summary(test$IBedRooms)# CLEAN
summary(test$IFloorAreaSqm)
summary(test$IRemainingLease)
summary(test$IPplStayinginFlatTypeinthisTown)
summary(test$IPopulationDensityinthisTown)

# A USer Defined Function to scale
UDF <- function(x) {
  (x -min(x))/ (max(x)- min(x))
}
trainDF <- as.data.frame(apply(trainDF, 2, UDF))
testDF <- as.data.frame(apply(test, 2, UDF))

# SPLItting the data.

index <- sample(nrow (trainDF), round(0.6 * nrow(trainDF)))

trainDF.wp <- trainDF[index,]
test.wp <- trainDF[-index,]

# MODEL



allVars <- colnames(trainDF)
predictorVars <- allVars[!allVars%in%"DResalePrice"]
predictorVars <- paste(predictorVars, collapse = "+")
form = as.formula(paste("DResalePrice~", predictorVars, collapse = "+"))

# Prediction Model
nn_model <- neuralnet(formula = form, trainDF.wp, hidden = c(4,2), linear.output = TRUE, stepmax=1e6)

# the fitted values i.e. weights
nn_model$net.result
plot(nn_model)

#PREDICTION

prediction1 <- compute(nn_model, test) #try predict works
#prediction1 <- predict(nn_model, test)
str(prediction1)

# UDF: Convert the scaled values to original 
UDF_2 <- function(prediction) {
  prediction1$net.result * (max(train_o$DResalePrice)-min(train_o$DResalePrice)) + min(train_o$DResalePrice)
}

ActualPrediction <-  prediction1$net.result * (max(train_o$DResalePrice)-min(train_o$DResalePrice)) + min(train_o$DResalePrice)

table(ActualPrediction)

#submit.df <- data.frame(Id = rep(1461:2919), SalePrice= ActualPrediction)
#write.csv(submit.df, file = "Submission_PredResul.csv", row.names = FALSE)

# Plot to show the correlation among the selected variables
#mydata <- train[, c("SUnitName", "IAgeofFlat","IBedRooms", "DResalePrice")]

#train_ <- round(cor(mydata),2)
#head(train_)

#library(reshape2)

#melted_train <- melt(train_)
#head(melted_train)

#library(ggplot2)
#ggplot(data = melted_train, aes(x=Var1, y=Var2, fill=value)) +   geom_tile()
#head(mydata)

#here the code for ensemble
#model with nnet
library(nnet)
model_nnet <- nnet(formula = form, data = trainDF, size = 10, maxit = 100000)
#nn_model <- neuralnet(formula = form, train.wp, hidden = c(4,2), linear.output = TRUE, stepmax=1e6)
#PREDICTION

prediction2 <- predict(model_nnet, test)
str(prediction2)


#try grnn model also
library(grnn)

size=nrow(trainDF)

length=ncol(trainDF)
model_grnn <- learn(trainDF, variable.column=length)
nn <- smooth(model_grnn, sigma = 0.5)

myControl <- trainControl(method='cv', number=3, returnResamp='none')


#Here is the one complicated part, instead of splitting the data into 2 parts of 
#train and test, we split the data into 3 parts: ensembleData, blenderData, and testingData:

library(caret)
#split <- floor(nrow(trainDF)/3)
#ensembleData <- trainDF[0:split,]
#blenderData <- trainDF[(split+1):(split*2),]
#testingData <- trainDF[(split*2+1):nrow(trainDF),]
ensembleData <- trainDF[0:300,]
blenderData <- trainDF[301:600,]
testingData <- trainDF[601:900,]
labelName <- 'DResalePrice'
predictors <- names(ensembleData)[names(ensembleData) != labelName]

blenderData$gbm_PROB <- predict(model_nnet, blenderData[,predictors])
testingData$gbm_PROB <- predict(object=model_nnet, testingData[,predictors])

#for Grnn

#blenderData$grnn_PROB <- compute(nn_model, test)
#testingData$grnn_PROB <- compute(nn_model, test)

for(i in 1:nrow(ensembleData))
{	
  vec <- as.matrix(ensembleData[i,])
  res <- guess(nn, vec)
  
  if(is.nan(res))
  {
    cat("Entry ",i," Generated NaN result!\n")
  }
  else
  {
    testingData$grnn_PROB[i] <- res
  }
}


for(i in 1:nrow(testingData))
{	
  vec <- as.matrix(testingData[i,])
  res <- guess(nn, vec)
  
  if(is.nan(res))
  {
    cat("Entry ",i," Generated NaN result!\n")
  }
  else
  {
    testingData$grnn_PROB[i] <- res
  }
}

predictors <- names(blenderData)[names(blenderData) != labelName]
final_blender_model <- train(blenderData[,predictors], blenderData[,labelName], method='gbm', trControl=myControl)

preds <- predict(object=final_blender_model, testingData[,predictors])
library(pROC)
auc <- roc(testingData[,labelName], preds)
print(auc$auc)

#model_gbm_finalModel <- 
